# Notebook 02: Visualise MRCP Features.
Visualise the MRCP features across EEG trials from the FeatureExtractor pipeline using Matplotlib and MNE-Python. This step ensures correct extraction and aids interpretation.
#### Rationale:
- Visual inspection validates that features (e.g. negative peak) align with expected MRCP components.
- MRCP onset typically occurs 1.0-2.0s before movement. 
- Deviations may signal preprocessing or extraction issues.
- Inter-subject and inter-trial variability is best captured graphically to detect artefacts or outliers.
- Ensures scientific and regulatory reproducibility (e.g., CE dossier, IEC 62304 validation) via transparent visual audit.

### Cell 1. Generate and Save One Synthetic MRCP Trial andFeature File.
Simulate one synthetic EEG trial with MRCP deflection. Extract channel-wise MRCP features (`area`, `peak`, `slope`). Save the result in NumPy and JSON formats to be used by downstream visualisation or classification modules.
#### Rationale:
- Produces a testable MRCP signal with synthetic deflection to support visual and quantitative validation.
- Mimics real MRCP dynamics using synthetic deflection in an otherwise noisy baseline.
- Ensures data reproducibility for visual inspection, audit trails, and machine learning training.
- Enables fully offline workflow for IEC 62304, AI Act, and GDPR-compliant BCI development.

In [5]:
# ========================================================================================
# Cell 1. Generate and save one synthetic MRCP trial and its extracted features.
# ========================================================================================

# Simulate one EEG trial with realistic movement-related deflection,
# extract MRCP features (area, peak, slope) using FeatureExtractor, and save both outputs
# to disk for visualisation and classifier input.

# Import numerical and I/O libraries used in signal processing and feature access.
import numpy as np              # Numpy handles EEG trial arrays (shape: [channels, samples]).
import matplotlib.pyplot as plt # Matplotlib enables graphical exploration of EEG features.
import json                     # JSON module allows loading structured MRCP feature dictionaries.
import os                       # OS module ensures platform-independent path handling.

# -- Signal Processing --
from bci_core.preprocessor import SignalPreprocessor
from bci_core.mrcp_feature_extractor import MRCPFeatureExtractor

# ----------------------------------------------------------------------------------------
# STEP 1: Define simulation parameters.
# ----------------------------------------------------------------------------------------

n_channels = 4                      # Number of EEG channels in the synthetic trial.
duration = 4.0                      # Trial in seconds.
sfreq = 250                         # Sampling frequency in Hz.
n_samples = int(duration * sfreq)   # Total number of time points per channel.

# ----------------------------------------------------------------------------------------
# STEP 2: Generate synthetic MRCP signal.
# ----------------------------------------------------------------------------------------

eeg = np.random.randn(n_channels, n_samples) * 5e-6     # Gaussian noise baseline.
mrcp_deflection = -np.exp(-np.linspace(0, duration, n_samples)) * 4e-6  # Simulated MRCP waveforms.
eeg += mrcp_deflection[None, :]     # Add deflection to all channels.

# ----------------------------------------------------------------------------------------
# STEP 3: Extract MRCP features using the shared FeatureExtractor.
# ----------------------------------------------------------------------------------------

extractor = MRCPFeatureExtractor(sfreq=sfreq) # Initialise the extractor.
features = extractor.extract_all(eeg)     # Compute all MRCP features. Keys per channel: 'area', 'peak', 'slope'.

# ----------------------------------------------------------------------------------------
# STEP 4: Save EEG signal and feature dictionary.
# ----------------------------------------------------------------------------------------

os.makedirs('../data/clean', exist_ok=True)  # Ensures EEG save folder exists.
os.makedirs('../data/feat',exist_ok=True)    # Ensures feature save folder exists.

np.save('../data/clean/trial_001.npy', eeg)  # Save EEG Matrix.
                                            # The .npy format is the standard binary file format in 
                                            # NumPy for persisting a single arbitrary NumPy array on 
                                            # disk. The format stores all of the shape and dtype 
                                            # information necessary to reconstruct the array correctly 
                                            # even on another machine with a different architecture.
with open('../data/feat/trial_001.json', 'w') as f:
    json.dump(features, f, indent=4)        # Save MRCP features as JSON.

# ----------------------------------------------------------------------------------------
# STEP 5: Confirm success.
# ----------------------------------------------------------------------------------------

# Convert string keys to integers to maintain channel index consistency.
features = {int(k): v for k, v in features.items()}

# Confirm correct struture and keys.
print('Synthetic trial and features saved successfully.')
print(f'EEG shape: {eeg.shape}')                 # Expected: (4, 1000).
print(f'Feature keys', list(features[0].keys())) # Should include: 'area', 'peak', 'slope'.

Synthetic trial and features saved successfully.
EEG shape: (4, 1000)
Feature keys ['area', 'peak', 'slope']


### Cell 2. Load Synthetic MRCP Trial and Extracted Features.
Load one preprocessed EEG trial and its associated MRCP feature dictionary from disk. The signal represents a simulated MRCP recording generated for development and validation purposes. Extracted features include negative peak amplitude, deflection slope, and total area under the curve.
#### Rationale:
- Enables visual inspection of one complete synthetic MRCP trial to verify preprocessing and feature alignment.
- Ensures file structire compliance (/data/clean and /data/feat) for reproducible research and future clinical validation.
- MRCP markers (e.g, negative peak, slope, area) are central to detecting movement intent in stroke rehabilitation systems.
- Confirming correct signal shape and feature presence is required under IEC 62304 traceability for SaMD (Service as Medical Device).

In [6]:
# ========================================================================================
# Cell 2. Load one synthetic EEG trial and its extracted MRCP features for inspection.
# ========================================================================================

# ----------------------------------------------------------------------------------------
# STEP 1: Define folders containing EEG trials and extracted feature files.
# ----------------------------------------------------------------------------------------

path_clean_trials = '../data/clean'  # Folder containing preprocessed EEG signals (.npy).
path_features = '../data/feat'       # Folder containing MRCP features extracted per trial (.json).

# ----------------------------------------------------------------------------------------
# STEP 2: Define the trial identifier to inspect.
# ----------------------------------------------------------------------------------------

trial_name = 'trial_001'    # Example trial: 'trial_001.npy' and 'trial_001.json'.

# ----------------------------------------------------------------------------------------
# STEP 3: Load EEG trial as 2D NumPy array.
#         Shape convention: (n_channels, n_samples), e.g. (4, 1000) for 4-channel EEG.
# ----------------------------------------------------------------------------------------

eeg = np.load(os.path.join(path_clean_trials, f'{trial_name}.npy')) # Load EEG data matrix from file.

# ----------------------------------------------------------------------------------------
# STEP 4: Load extracted MRCP features for the same trial.
# ----------------------------------------------------------------------------------------

with open(os.path.join(path_features, f'{trial_name}.json')) as f:
    features = json.load(f)

# ----------------------------------------------------------------------------------------
# STEP 5: Print summary to confirm successful loading.
# ----------------------------------------------------------------------------------------

# Convert string keys to intergers to maintain channel index consistency.
features = {int(k): v for k, v in features.items()}

print(f'Loaded EEG shape: {eeg.shape}')         # Expected: e.g. (4, 1000): 4 EEG channels x 1000 time points.
print('Extracted Features:', features.keys())   # Expected: 'area', 'peak', 'slope' per channel.
print('Feature keys (per channel):', features[0].keys()) # Expected: dict_keys(['area', 'peak', 'slope']).

Loaded EEG shape: (4, 1000)
Extracted Features: dict_keys([0, 1, 2, 3])
Feature keys (per channel): dict_keys(['area', 'peak', 'slope'])


### Cell 3. Plot MRCP Waveforms and Annotate Extracted Features.
Plot each EEG channel's waveform and visually overlay the extracted MRCP features, including the negative `peak`, `slope` reference point, and shaded `area`. This step confirms alignment between raw data and extracted features.
#### Rationale:
- Helps validate the correctness of your `MRCPFeatureExtractor`.
- Ensures that clinical features (like negative peak latency or slope timing) are visually interpretable.
- Supports regularly traceability by linking signal to extract metadata.
- Serves as visual debug tool for real vs. synthetic MRCP trials.

In [ ]:
# ========================================================================================
# Cell 3. Plot MRCP waveforms for each channel with annotated features.
# ========================================================================================

# Create time axis in second (x-axis for plots).
time = np.linspace(0, duration, eeg.shape[1]) # From 0 to 4.0 s, evenly spaced.

# Create figure: one subplot per EEG channel.
# - n_channels, 1: Creates a vertical stack of n_channels subplots (e.g., 4 channels: 4 rows, 1 column). 
# Each will show one EEG channel.figsize=(10, 8).
# - figsize=(10, 8): Specifies the figure size in inches: 10 wide × 8 tall. A larger height makes room 
#   for each EEG trace.
# - sharex=True: All subplots share the same x-axis, which represents time in EEG. This is crucial in BCI, 
# so you can align activity across channels.
fig, axes = plt.subplots(n_channels, 1, figsize=(10, 8), sharex=True)
fig.suptitle('Synthetic MRCP Waveforms with Extracted Features', fontsize=14)
